# Statistics Explained  articles and matches with Eurostat's Concepts and Definitions Database

## Objective: to build a common vocabulary and construct "profiles" of the SE articles

In [1]:
import re
import pandas as pd
import spacy
import sys
from collections import Counter

## Run to install the language library, then comment-out
#!{sys.executable} -m spacy download en_core_web_lg

nlp = spacy.load('en_core_web_lg')
nlp.max_length = 1500000
print('Finished loading.')



Finished loading.


In [2]:
from datetime import datetime

def file_name(pre,ext):
    current_time = datetime.now() 
    return pre + '_'+ str(current_time.month)+ '_' + str(current_time.day) + \
                 '_' + str(current_time.hour)+ '_' + str(current_time.minute)  +'.'+ext

In [3]:
import pyodbc
c = pyodbc.connect('DSN=Virtuoso All;DBA=ESTAT;UID=kimon;PWD=RkhvQYZ442e2JVXLHdtW')
cursor = c.cursor()

In [4]:

#import unicodedata as ud

def clean(x, quotes=True):
    if pd.isnull(x): return x  
    x = x.strip()
    
    ## make letter-question mark-letter -> letter-quote-space-letter !!! but NOT in the lists of URLs!!!
    if quotes:
        x = re.sub(r'([A-Za-z])\?([A-Za-z])','\\1\' \\2',x) 
    
    ## make letter-question mark-space lower case letter letter-quote-space letter
    x = re.sub(r'([A-Za-z])\? ([a-z])','\\1\' \\2',x) 

    ## delete ,000 commas in numbers    
    x = re.sub(r'\b(\d+),(\d+)\b','\\1\\2',x) ## CORRECTED
    
    ## delete  000 spaces in numbers
    x = re.sub(r'\b(\d+) (\d+)\b','\\1\\2',x) ## CORRECTED
    
    ## remove more than one spaces
    x = re.sub(r' +', ' ',x)
    
    ## remove start and end spaces
    x = re.sub(r'^ +| +$', '',x,flags=re.MULTILINE) 
    
    ## space-comma -> comma
    x = re.sub(r' \,',',',x)
    
    ## space-dot -> dot
    x = re.sub(r' \.','.',x)
    
    x = re.sub(r'â.{2}',"'",x) ### !!! NEW: single quotes are read as: âXX
    
    #x = x.encode('latin1').decode('utf-8') ## â\x80\x99
    #x = ud.normalize('NFKD',x).encode('ascii', 'ignore').decode()
    
    return x

### Statistics explained articles

* IDs, titles from dat_link_info, with resource_information_id=1, i.e. Eurostat (see ESTAT.V1.mod_resource_information) and matching IDs from dat_article.
* Carry out data cleansing on titles.


In [5]:
SQLCommand = """SELECT id, title 
                FROM ESTAT.V1.dat_link_info 
                WHERE resource_information_id=1 AND id IN (SELECT id FROM ESTAT.V1.dat_article) """

SE_df = pd.read_sql(SQLCommand,c)

SE_df['title'] = SE_df['title'].apply(clean)
SE_df.head(5)


,id,title
0,7,Accidents at work statistics
1,13,National accounts and GDP
2,16,Railway safety statistics in the EU
3,17,Railway freight transport statistics
4,18,Railway passenger transport statistics - quart...


### Add paragraphs titles and contents

* From dat_article_paragraph with abstract=0 (i.e. "no").
* Match article_id from dat_article_paragraph with id from dat_article.
* Carry out data cleansing on titles and paragraph contents.

In [6]:
SQLCommand = """SELECT article_id, title, content 
                FROM ESTAT.V1.dat_article_paragraph
                WHERE abstract=0 AND article_id IN (SELECT id FROM ESTAT.V1.dat_article) """

add_content = pd.read_sql(SQLCommand,c)
add_content['title'] = add_content['title'].apply(clean)
add_content['content'] = add_content['content'].apply(clean)
add_content

,article_id,title,content
0,2905,Absences from work sharply increase in first h...,Absences from work recorded unprecedented high...
1,2905,Absences: 9.5 % of employment in Q4 2019 and 1...,The article's next figure (Figure 4) compares ...
2,2905,Higher share of absences from work among women...,"Considering all four quarters of 2020, the sha..."
3,2905,Absences from work due to own illness or disab...,"From Q4 2019 to Q4 2020, the number of people ..."
4,2905,Absences from work due to holidays,"Expressed as a share of employed people, absen..."
...,...,...,...
3854,10539,General presentation and definition,Scope of asylum statistics and Dublin statisti...
3855,10539,Methodological aspects in asylum statistics,Annual aggregate of the number of asylum appli...
3856,10539,Methodological aspects in Dublin statistics,Asymmetries For most of the collected Dublin s...
3857,10539,What questions can or cannot be answered with ...,How many asylum seekers are entering EU Member...


### Aggregate above paragraph titles and contents  from SE articles paragraphs by article id

* Create a column _raw content_ which gathers all paragraph titles and contents in one text per article.

In [7]:
add_content_grouped = add_content.groupby(['article_id'])[['title','content']].aggregate(lambda x: list(x))
add_content_grouped.reset_index(drop=False, inplace=True)
for i in range(len(add_content_grouped)):
    add_content_grouped.loc[i,'raw content'] = ''
    for (a,b) in zip(add_content_grouped.loc[i,'title'],add_content_grouped.loc[i,'content']):
        add_content_grouped.loc[i,'raw content'] += ' '+a + ' ' + b
add_content_grouped = add_content_grouped[['article_id','raw content']]    

add_content_grouped

,article_id,raw content
0,7,"Number of accidents In 2018, there were 3.1 m..."
1,13,Developments for GDP in the EU-27: growth sin...
2,16,Fall in the number of railway accidents 9 % f...
3,17,Downturn for EU transport performance in 2019...
4,18,Rail passenger transport performance continue...
...,...,...
860,10456,Problem After successfully identifying and jo...
861,10470,"Problem In France, there was significant room..."
862,10506,General overview Nine PEEIs concern short-ter...
863,10531,What are administrative sources? The term 'ad...


### Merge raw content of SE articles with main file

* Also, add title to definition.

In [8]:
SE_df = pd.merge(SE_df,add_content_grouped,left_on='id',right_on='article_id',how='inner')
SE_df.drop(['article_id'],axis=1,inplace=True)

SE_df['raw content'] = SE_df['title'] +'. '+SE_df['raw content']

SE_df.head(5)

,id,title,raw content
0,7,Accidents at work statistics,Accidents at work statistics. Number of accid...
1,13,National accounts and GDP,National accounts and GDP. Developments for G...
2,16,Railway safety statistics in the EU,Railway safety statistics in the EU. Fall in ...
3,17,Railway freight transport statistics,Railway freight transport statistics. Downtur...
4,18,Railway passenger transport statistics - quart...,Railway passenger transport statistics - quart...


### Lemmatize 'raw content'

* NLTK seems to be better than Spacy in lemmatization. Convert to lower-case before.

In [9]:
import nltk

w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
lemmatizer = nltk.stem.WordNetLemmatizer()

def lemmatize_text(text):
    return [lemmatizer.lemmatize(w) for w in w_tokenizer.tokenize(text)]

SE_df['raw content'] = SE_df['raw content'].apply(lambda x: x.lower())
SE_df['raw content']= SE_df['raw content'].apply(lemmatize_text)
SE_df['raw content']= [' '.join(map(str, l)) for l in SE_df['raw content']]
SE_df['raw content'] = SE_df['raw content'].apply(lambda x: x.upper())
SE_df


,id,title,raw content
0,7,Accidents at work statistics,ACCIDENT AT WORK STATISTICS. NUMBER OF ACCIDEN...
1,13,National accounts and GDP,NATIONAL ACCOUNT AND GDP. DEVELOPMENT FOR GDP ...
2,16,Railway safety statistics in the EU,RAILWAY SAFETY STATISTIC IN THE EU. FALL IN TH...
3,17,Railway freight transport statistics,RAILWAY FREIGHT TRANSPORT STATISTICS. DOWNTURN...
4,18,Railway passenger transport statistics - quart...,RAILWAY PASSENGER TRANSPORT STATISTIC - QUARTE...
...,...,...,...
860,10456,"Merging statistics and geospatial information,...","MERGING STATISTIC AND GEOSPATIAL INFORMATION, ..."
861,10470,"Merging statistics and geospatial information,...","MERGING STATISTIC AND GEOSPATIAL INFORMATION, ..."
862,10506,Methods for compiling PEEIs in short-term busi...,METHOD FOR COMPILING PEEIS IN SHORT-TERM BUSIN...
863,10531,Building the System of National Accounts - adm...,BUILDING THE SYSTEM OF NATIONAL ACCOUNT - ADMI...


### Eurostat's Concepts and Definitions Database

* Download from database.

In [10]:
SQLCommand = """SELECT id, code_id, term
                FROM ESTAT.V1.dat_estat_glossary 
             """


concepts_df = pd.read_sql(SQLCommand,c)
concepts_df

concepts_df

,id,code_id,term
0,1,12789,"(n,k) rule"
1,2,12799,"(p,q) rule"
2,3,19247,Âµ-ARGUS
3,4,5545,Abandoned wine-growing area
4,5,20003,Abduction by a legal guardian
...,...,...,...
11215,11216,4277,Value adjustments on investments
11216,11217,4284,Value of transactions between affiliated enter...
11217,11218,4285,Value re-adjustments on investments
11218,11219,19099,Widened agricultural census


### Lemmatize terms

* Convert to lower-case first.

In [11]:
concepts_df['lemmatized_term'] = concepts_df['term'].apply(lambda x: x.lower())
concepts_df['lemmatized_term']= concepts_df['lemmatized_term'].apply(lemmatize_text)
concepts_df['lemmatized_term']= [' '.join(map(str, l)) for l in concepts_df['lemmatized_term']]
concepts_df['lemmatized_term'] = concepts_df['lemmatized_term'].apply(lambda x: x.upper())
concepts_df

,id,code_id,term,lemmatized_term
0,1,12789,"(n,k) rule","(N,K) RULE"
1,2,12799,"(p,q) rule","(P,Q) RULE"
2,3,19247,Âµ-ARGUS,ÂΜ-ARGUS
3,4,5545,Abandoned wine-growing area,ABANDONED WINE-GROWING AREA
4,5,20003,Abduction by a legal guardian,ABDUCTION BY A LEGAL GUARDIAN
...,...,...,...,...
11215,11216,4277,Value adjustments on investments,VALUE ADJUSTMENT ON INVESTMENT
11216,11217,4284,Value of transactions between affiliated enter...,VALUE OF TRANSACTION BETWEEN AFFILIATED ENTERP...
11217,11218,4285,Value re-adjustments on investments,VALUE RE-ADJUSTMENTS ON INVESTMENT
11218,11219,19099,Widened agricultural census,WIDENED AGRICULTURAL CENSUS


### Prepare Spacy's PhraseMatcher by building a custom vocabulary from Eurostat's database of concepts and definitions (column 'lemmatized_term')


In [12]:
from spacy.matcher import PhraseMatcher

matcher = PhraseMatcher(nlp.vocab)
terms = concepts_df['lemmatized_term'].values.tolist()

patterns = [nlp.make_doc(text) for text in terms]
matcher.add("TerminologyList", patterns)

### Apply PhraseMatcher

* Collect results per SE article ('doc_id') in a dataframe 'res'. Ignore matches with 2 words or less.
* Depending on the length of a match: columns '3-Phrases', '4-Phrases', '5-and-above-Phrases'. These will contain dictionaries with the matched lemmatized terms and their counts, in descending order of counts.
* Column 'Terms' has a dictionary with the corresponding **original terms** in Eurostat's database of concepts and definitions and their counts in the matches.

In [13]:
res = pd.DataFrame(index=range(len(SE_df)))
res['3-Phrases']=[[] for i in range(len(SE_df))]
res['4-Phrases']=[[] for i in range(len(SE_df))]
res['5-and-above-Phrases']=[[] for i in range(len(SE_df))]
res['Terms']=[dict() for i in range(len(SE_df))]
docs=nlp.pipe(SE_df['raw content'])
for (i,doc) in enumerate(docs):
    print(i)
    for sent in doc.sents:
        matches = matcher(sent)
        for match_id, start, end in matches:
            span = doc[start:end]
            n_words = len(span.text.split(' '))
            if n_words >= 3:
                doc_id = SE_df.loc[i,'id']
                idx = concepts_df.index[concepts_df['lemmatized_term'].str.contains(span.text,regex=False)].tolist()
                print(i,SE_df.loc[i,'title'],len(sent.text),'>',n_words,span.text,idx)
                res.loc[i,'doc_id']=doc_id
                for elem in concepts_df.loc[idx,'term'].values.tolist():
                    if elem in res.loc[i,'Terms'].keys():
                        res.loc[i,'Terms'][elem] +=1
                    else:
                        res.loc[i,'Terms'][elem] =1
                #res.loc[i,'Terms'].append(concepts_df.loc[idx,'term'].values.tolist())
                if n_words == 3:
                    res.loc[i,'3-Phrases'].append(span.text)
                elif n_words == 4:
                    res.loc[i,'4-Phrases'].append(span.text)
                else:
                    res.loc[i,'5-and-above-Phrases'].append(span.text)
                    
                    


0
0 Accidents at work statistics 28 > 3 ACCIDENT AT WORK [35]
0 Accidents at work statistics 145 > 3 ACCIDENT AT WORK [35]
0 Accidents at work statistics 146 > 3 ACCIDENT AT WORK [35]
0 Accidents at work statistics 73 > 3 ACCIDENT AT WORK [35]
0 Accidents at work statistics 104 > 3 ACCIDENT AT WORK [35]
0 Accidents at work statistics 218 > 3 ACCIDENT AT WORK [35]
0 Accidents at work statistics 161 > 3 ACCIDENT AT WORK [35]
0 Accidents at work statistics 197 > 3 ACCIDENT AT WORK [35]
0 Accidents at work statistics 197 > 3 ACCIDENT AT WORK [35]
0 Accidents at work statistics 92 > 3 ACCIDENT AT WORK [35]
0 Accidents at work statistics 53 > 3 ACCIDENT AT WORK [35]
0 Accidents at work statistics 278 > 3 ACCIDENT AT WORK [35]
0 Accidents at work statistics 278 > 3 ACCIDENT AT WORK [35]
0 Accidents at work statistics 379 > 3 ACCIDENT AT WORK [35]
0 Accidents at work statistics 324 > 4 HEALTH AND SOCIAL WORK [4344]
0 Accidents at work statistics 145 > 3 TYPE OF INJURY [10482]
0 Accidents at wo

11 Adult education survey - methodology 161 > 3 DEGREE OF URBANISATION [2376]
12
12 Adult learning statistics - characteristics of education and training 231 > 8 PLANNED PERIOD OF TRAINING, INSTRUCTION OR PRACTICAL EXPERIENCE [7479]
12 Adult learning statistics - characteristics of education and training 231 > 3 PLACE OF WORK [5583, 7470]
12 Adult learning statistics - characteristics of education and training 195 > 3 LEVEL OF EDUCATION [4404, 5446, 5451]
13
13 SDG 4 - Quality education 186 > 5 EARLY CHILDHOOD EDUCATION AND CARE [2737]
13 SDG 4 - Quality education 76 > 5 EARLY CHILDHOOD EDUCATION AND CARE [2737]
13 SDG 4 - Quality education 141 > 5 EARLY CHILDHOOD EDUCATION AND CARE [2737]
13 SDG 4 - Quality education 175 > 3 LEVEL OF EDUCATION [4404, 5446, 5451]
13 SDG 4 - Quality education 195 > 5 PROGRAMME FOR INTERNATIONAL STUDENT ASSESSMENT [7910]
14
14 Agriculture statistics at regional level 141 > 3 UTILISED AGRICULTURAL AREA [10695, 10696, 10697, 10698, 10699]
14 Agriculture st

32 Living conditions in Europe - income distribution and income inequality 161 > 3 EQUIVALISED DISPOSABLE INCOME [3135]
32 Living conditions in Europe - income distribution and income inequality 208 > 3 EQUIVALISED DISPOSABLE INCOME [3135]
32 Living conditions in Europe - income distribution and income inequality 79 > 3 EQUIVALISED DISPOSABLE INCOME [3135]
32 Living conditions in Europe - income distribution and income inequality 215 > 3 EQUIVALISED DISPOSABLE INCOME [3135]
32 Living conditions in Europe - income distribution and income inequality 292 > 3 EQUIVALISED DISPOSABLE INCOME [3135]
32 Living conditions in Europe - income distribution and income inequality 227 > 3 EQUIVALISED DISPOSABLE INCOME [3135]
32 Living conditions in Europe - income distribution and income inequality 168 > 3 EQUIVALISED DISPOSABLE INCOME [3135]
33
33 International trade in goods 148 > 4 AVERAGE ANNUAL GROWTH RATE [620, 621]
33 International trade in goods 148 > 3 ANNUAL GROWTH RATE [418, 428, 620, 621]


45 Environmental accounts - establishing the links between the environment and the economy 44 > 5 ENVIRONMENTAL GOOD AND SERVICE SECTOR [3087, 3088]
45 Environmental accounts - establishing the links between the environment and the economy 44 > 6 ENVIRONMENTAL GOOD AND SERVICE SECTOR (EGSS) [3088]
45 Environmental accounts - establishing the links between the environment and the economy 113 > 3 GROSS VALUE ADDED [4257, 4258, 4259, 4260, 4261, 4262]
45 Environmental accounts - establishing the links between the environment and the economy 47 > 3 ENVIRONMENTAL PROTECTION EXPENDITURE [3093, 3094]
45 Environmental accounts - establishing the links between the environment and the economy 47 > 4 ENVIRONMENTAL PROTECTION EXPENDITURE ACCOUNT [3094]
45 Environmental accounts - establishing the links between the environment and the economy 221 > 4 GROSS FIXED CAPITAL FORMATION [4211, 4212, 4213, 4214, 4215, 4216, 4217, 10718, 11059]
45 Environmental accounts - establishing the links between the 

56 Government finance statistics - quarterly data 108 > 4 FINANCIAL ASSET AND LIABILITY [3493, 7014]
56 Government finance statistics - quarterly data 78 > 4 FINANCIAL ASSET AND LIABILITY [3493, 7014]
56 Government finance statistics - quarterly data 128 > 3 CURRENCY AND DEPOSIT [2012, 2013, 2014]
57
57 Structure of government debt 251 > 3 GENERAL GOVERNMENT SECTOR [3912]
57 Structure of government debt 251 > 3 SOCIAL SECURITY FUND [9293, 9294, 9295]
57 Structure of government debt 230 > 3 SOCIAL SECURITY FUND [9293, 9294, 9295]
57 Structure of government debt 168 > 3 CURRENCY AND DEPOSIT [2012, 2013, 2014]
57 Structure of government debt 96 > 3 CURRENCY AND DEPOSIT [2012, 2013, 2014]
57 Structure of government debt 233 > 3 CURRENCY AND DEPOSIT [2012, 2013, 2014]
57 Structure of government debt 304 > 4 REST OF THE WORLD [8619, 8620, 8621, 8622, 8623, 8624]
57 Structure of government debt 148 > 3 FINANCIAL CORPORATION SECTOR [3505, 3506, 6459]
57 Structure of government debt 63 > 4 REST

...

825 Globalisation patterns in trade and investment - introduction 503 > 4 EUROPEAN STATISTICAL SYSTEM (ESS) [3230, 3231]
825 Globalisation patterns in trade and investment - introduction 503 > 3 FOREIGN DIRECT INVESTMENT [3658, 3659, 3660, 3661, 3662, 3663, 3664, 3665, 3666]
825 Globalisation patterns in trade and investment - introduction 503 > 3 STRUCTURAL BUSINESS STATISTIC [9686]
825 Globalisation patterns in trade and investment - introduction 286 > 3 DATA COLLECTION METHOD [2122]
825 Globalisation patterns in trade and investment - introduction 764 > 3 SPECIAL PURPOSE ENTITY [9369, 9370, 9371, 9372]
825 Globalisation patterns in trade and investment - introduction 332 > 3 EUROPEAN CENTRAL BANK [3203]
825 Globalisation patterns in trade and investment - introduction 332 > 3 INTERNATIONAL MONETARY FUND [5074, 5075]
825 Globalisation patterns in trade and investment - introduction 242 > 4 REST OF THE WORLD [8619, 8620, 8621, 8622, 8623, 8624]
825 Globalisation patterns in trade and 

834 EU statistics on income and living conditions (EU-SILC) methodology ' data quality 259 > 3 LOGISTIC REGRESSION MODEL [5599]
834 EU statistics on income and living conditions (EU-SILC) methodology ' data quality 149 > 3 ITEM NON-RESPONSE RATE [5219]
834 EU statistics on income and living conditions (EU-SILC) methodology ' data quality 115 > 3 PARTIAL UNIT NON-RESPONSE [7181]
834 EU statistics on income and living conditions (EU-SILC) methodology ' data quality 340 > 3 PARTIAL UNIT NON-RESPONSE [7181]
834 EU statistics on income and living conditions (EU-SILC) methodology ' data quality 98 > 3 LEVEL OF DATA [5450]
834 EU statistics on income and living conditions (EU-SILC) methodology ' data quality 184 > 4 EUROPEAN STATISTICAL SYSTEM (ESS) [3230, 3231]
834 EU statistics on income and living conditions (EU-SILC) methodology ' data quality 266 > 3 COST AND BURDEN [1821, 1822, 1823]
835
835 EU statistics on income and living conditions (EU-SILC) methodology - people at risk of poverty 

842
842 EU labour force survey - documentation 231 > 3 DEGREE OF URBANISATION [2376]
842 EU labour force survey - documentation 83 > 6 NOMENCLATURE OF TERRITORIAL UNIT FOR STATISTIC [6400]
842 EU labour force survey - documentation 22 > 3 DEGREE OF URBANISATION [2376]
842 EU labour force survey - documentation 62 > 3 DEGREE OF URBANISATION [2376]
842 EU labour force survey - documentation 105 > 3 DEGREE OF URBANISATION [2376]
842 EU labour force survey - documentation 62 > 6 INTERNATIONAL STANDARD CLASSIFICATION OF EDUCATION (ISCED) [5087, 5088]
842 EU labour force survey - documentation 199 > 3 FIELD OF EDUCATION [3431]
842 EU labour force survey - documentation 199 > 3 FIELD OF EDUCATION [3431]
842 EU labour force survey - documentation 127 > 3 FIELD OF EDUCATION [3431]
842 EU labour force survey - documentation 188 > 5 INTERNATIONAL STANDARD CLASSIFICATION OF OCCUPATION [5089]
842 EU labour force survey - documentation 101 > 10 STATISTICAL CLASSIFICATION OF ECONOMIC ACTIVITY IN THE 

854 Disability statistics background - Labour force survey - non-response analysis 110 > 3 DEGREE OF URBANISATION [2376]
854 Disability statistics background - Labour force survey - non-response analysis 172 > 3 LEVEL OF EDUCATION [4404, 5446, 5451]
854 Disability statistics background - Labour force survey - non-response analysis 217 > 3 LEVEL OF EDUCATION [4404, 5446, 5451]
855
855 Continuing Vocational Training Survey (CVTS) methodology 57 > 3 CONTINUING VOCATIONAL TRAINING [1721, 1722, 1723, 1724, 6908]
855 Continuing Vocational Training Survey (CVTS) methodology 57 > 5 CONTINUING VOCATIONAL TRAINING SURVEY (CVTS) [1723]
855 Continuing Vocational Training Survey (CVTS) methodology 147 > 10 STATISTICAL CLASSIFICATION OF ECONOMIC ACTIVITY IN THE EUROPEAN COMMUNITY (NACE) [9481, 9482]
855 Continuing Vocational Training Survey (CVTS) methodology 178 > 4 HEALTH AND SOCIAL WORK [4344]
855 Continuing Vocational Training Survey (CVTS) methodology 442 > 3 CONTINUING VOCATIONAL TRAINING [172

863 Building the System of National Accounts - administrative sources 164 > 3 TAX ON PRODUCT [7002, 7003, 9976, 9977, 9978, 9979, 9980, 9981, 9982]
863 Building the System of National Accounts - administrative sources 164 > 3 SUBSIDY ON PRODUCT [6993, 6994, 6995, 6996, 9741, 9742, 9743, 9744, 9745, 9746, 9747, 9956]
863 Building the System of National Accounts - administrative sources 66 > 3 RESEARCH AND DEVELOPMENT [6707, 8551, 8552, 8553, 8554, 8555]
863 Building the System of National Accounts - administrative sources 115 > 3 WITHDRAWAL FROM STOCK [10926]
863 Building the System of National Accounts - administrative sources 141 > 3 WITHDRAWAL FROM STOCK [10926]
863 Building the System of National Accounts - administrative sources 82 > 3 WITHDRAWAL FROM STOCK [10926]
863 Building the System of National Accounts - administrative sources 84 > 3 WITHDRAWAL FROM STOCK [10926]
863 Building the System of National Accounts - administrative sources 164 > 4 CONSUMPTION OF FIXED CAPITAL [1684,

In [14]:
res['3-Phrases']= res['3-Phrases'].apply(lambda x: dict(Counter(x).most_common()))
res['4-Phrases']= res['4-Phrases'].apply(lambda x: dict(Counter(x).most_common()))
res['5-and-above-Phrases']= res['5-and-above-Phrases'].apply(lambda x: dict(Counter(x).most_common()))



In [15]:
res=pd.merge(SE_df[['id','title']],res,left_on='id',right_on='doc_id',how='left')
res.drop(columns=['doc_id'],inplace=True)
res

,id,title,3-Phrases,4-Phrases,5-and-above-Phrases,Terms
0,7,Accidents at work statistics,"{'ACCIDENT AT WORK': 20, 'TYPE OF INJURY': 3, ...",{'HEALTH AND SOCIAL WORK': 1},{},"{'Accident at work': 20, 'Health and social wo..."
1,13,National accounts and GDP,"{'RATE OF CHANGE': 15, 'GROSS VALUE ADDED': 6,...","{'HEALTH AND SOCIAL WORK': 3, 'GROSS FIXED CAP...","{'TAX ON PRODUCTION AND IMPORT': 2, 'EXTERNAL ...","{'Annualised rate of change': 15, 'Rates of ch..."
2,16,Railway safety statistics in the EU,"{'PERSON SERIOUSLY INJURED': 6, 'LEVEL CROSSIN...",{'FIRE IN ROLLING STOCK': 1},{'ACCIDENT TO PERSON CAUSED BY ROLLING STOCK I...,"{'Level crossing accidents': 3, 'Fires in roll..."
3,17,Railway freight transport statistics,NaN,NaN,NaN,NaN
4,18,Railway passenger transport statistics - quart...,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
860,10456,"Merging statistics and geospatial information,...",{'GEOGRAPHIC INFORMATION SYSTEM': 1},{'GEOGRAPHIC INFORMATION SYSTEM (GIS)': 1},{},"{'Geographic information system': 1, 'Geograph..."
861,10470,"Merging statistics and geospatial information,...","{'STATISTICAL BUSINESS REGISTER': 3, 'PRODUCTI...",{'POPULATION AND HOUSING CENSUS': 1},{},"{'Statistical business register': 3, 'Statisti..."
862,10506,Methods for compiling PEEIs in short-term busi...,"{'PRODUCER PRICE INDEX': 3, 'VOLUME OF SALE': ...","{'POSTAL AND COURIER SERVICE': 1, 'FREQUENCY O...",{},"{'Volume of sales': 1, 'Industrial producer pr..."
863,10531,Building the System of National Accounts - adm...,"{'GROSS VALUE ADDED': 4, 'WITHDRAWAL FROM STOC...","{'REST OF THE WORLD': 5, 'CONSUMPTION OF FIXED...",{'FINANCIAL INTERMEDIATION SERVICE INDIRECTLY ...,"{'Value added tax': 2, 'Value added tax (VAT)'..."


In [16]:
outfile = file_name('Phrase_Matcher_SE_Eurostat','xlsx')
res.to_excel(outfile)
